INSTALL THE APPROPRIATE VERSION OF THE OPENCV LIBRARY TO ACCESS SIFT AND ORB ALGORITHMS

In [0]:
!pip install opencv-python==3.3.0.10 opencv-contrib-python==3.3.0.10

In [0]:
import numpy as np
import math 
import os 
import cv2
import matplotlib.pyplot as plt

DRAFT FUNCTIONS MADE TO ANALYSE THE OUTPUT FOR EACH IMAGE

In [0]:
#function for finding the top 5 images in the dataset for a test image
#INPUTS: 1.TEST IMAGE NAME 2.VALUE OF K 3.IMG_LIST 4. PATH OF THE IMAGES 5.MAXIMUM KEY POINTS TO BE USED

def img_correspondence(img_name,k,img_list,img_path,max_keypoints):
    #img_list.remove(img_name)
    img_test = cv2.imread(img_path + img_name)  #READ THE TEST IMAGE 
    img_test = cv2.cvtColor(img_test,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create(max_keypoints) #INITIALIZING SIFT OR ORB
    _, des_test = sift.detectAndCompute(img_test,None) #CALCUATING THE DESCRIPTORS
    euclidean_distances=np.zeros((6,))                 #THE 6 LEAST SUM OF EUCLIDEAN DISTANCES BETWEEN TEST AND COMPARING IMAGES
    top_images = np.zeros((6,))                        # THE TOP 6 IMAGES 
    
    for i in range(len(img_list)):                      # COMPARING IMAGES
        img = cv2.imread(img_path+img_list[i])
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        _,des = sift.detectAndCompute(img,None)         # DESCRIPTORS FOR THE COMPARING IMAGE
        top_k_matches = np.zeros((k,))
        if k=='large':
            top_k_matches = np.zeros((des.shape[0],1))
        
        for p in range(des_test.shape[0]):              # CALCULATING THE EUCLIDEAN DISTANCES BETWEEN EACH OF THE DESCRIPTORS
            for q in range(des.shape[0]):
                dist = np.linalg.norm(des[q,:]-des_test[p,:])
                if q==0:                                
                    dist_min = dist
                elif q>0 and dist<dist_min:
                    dist_min = dist
                if p<k:
                    top_k_matches[p]=dist_min           
                else:
                    if dist_min<top_k_matches[np.argmax(top_k_matches)]:
                        top_k_matches[np.argmax(top_k_matches)] = dist_min #CALCULATING THE TOP K MATCHES
        if i<6:                                             #CALCULATING THE SUM OF THE EUCLIDEAN DISTANCES
            euclidean_distances[i] = np.sum(top_k_matches)
            top_images[i] = i 
        else:
            if np.sum(top_k_matches)<euclidean_distances[np.argmax(euclidean_distances)]:
                top_images[np.argmax(euclidean_distances)] = i
                euclidean_distances[np.argmax(euclidean_distances)] = np.sum(top_k_matches)
    
    matched_images = []             #GETTING THE MATCHED IMAGES
    for r in range(6):
        matched_images.append(img_list[int(top_images[r])])
    return matched_images

In [0]:
# PLOTTING THE ACCURACY GRAPHS BASED ON THE VALUES OF K
def accuracy_graphs(img_list,k_list,img_path,max_keypoints):
    accuracies = np.zeros((len(k_list),len(img_list)))
    for j in range(len(k_list)):
        print(k_list[j])
        for i in range(len(img_list)):
            print('i=',img_list[i])
            matched_images = img_correspondence(img_list[i],k_list[j],img_list,img_path,max_keypoints)
            print('matched images=',matched_images)
            count = 0
            for c in range(len(matched_images)):
                if matched_images[c][0:2] == img_list[i][0:2] and matched_images[c]!=img_list[i]:
                    count+=1
            accuracies[j,i] = (count/5)*100
            print(accuracies[j,i])
    accuracies_avg = (np.sum(accuracies,axis=1))/len(img_list)
    print(accuracies_avg)
    plt.plot(k_list,accuracies_avg)
    plt.xlabel('k_values')
    plt.ylabel('accuracies')
    plt.title('average accuracies based on the k values using SIFT')
    plt.savefig('abc.png')    

MAIN FUNCTIONS USED TO DO THE ASSIGNMENT

In [0]:
##THIS IS THE FUNCTION FOR CALCULATING THE EUCLIDEAN DISTANCES BETWEEN THE IMAGE KEYPOINTS

#INPUTS FOR THE FUNCTION : 1.LIST OF THE IMAGE NAMES 2.PATH OF THE INPUT IMAGES 3. TYPE OF ALGORITHM USED: SIFT (OR) ORB
def euclidean_distances(img_list,img_path,algo): 
    euc_dict={}                                             #OUTPUT INITIALIZATION
    if algo=='SIFT':                                        #DEPENDING ON THE ALGORITHM
        sift = cv2.xfeatures2d.SIFT_create(500)                #CREATION OF SIFT
    elif algo== 'ORB':
        orb = cv2.ORB_create(500)                           #CREATION OF ORB
    for i in range(len(img_list)):                          # FOR EVERY TEST IMAGE
        print(i)                                             
        img_test = cv2.imread(img_path+img_list[i])         #READ THE TEST IMAGE
        img_test = cv2.cvtColor(img_test,cv2.COLOR_BGR2GRAY) #CONVERT TO GRAYSCALE
        if algo=='SIFT':
            _,des_test = sift.detectAndCompute(img_test,None) #FINDING THE IMAGE KEYPOINT DESCRIPTORS
        elif algo=='ORB':
            _,des_test = orb.detectAndCompute(img_test,None)
        #COMPARING WITH REMAINING IMAGES IN THE DATASET
        for j in range(len(img_list)):
            if img_list[i]!=img_list[j] and ((img_list[i][0:5],img_list[j][0:5]) and (img_list[j][0:5],img_list[i][0:5]) not in list(euc_dict.keys())): #AVOIDING REPITION IN CALCULATION OF EUCLIDEAN DISTANCES
                    img_j = cv2.imread(img_path+img_list[j]) #READING THE COMPARING IMAGE
                    img_j = cv2.cvtColor(img_j,cv2.COLOR_BGR2GRAY) #CONVERTING TO GRAYSCALE
                    if algo=='SIFT':
                        _,des_j = sift.detectAndCompute(img_j,None)
                    elif algo=='ORB':
                        _,des_j = orb.detectAndCompute(img_j,None)
                    distances = [] # TO STORE ALL THE DISTANCES IN A LIST
                    for p in range(des_test.shape[0]):
                        for q in range(des_j.shape[0]):
                            distances.append(np.linalg.norm(des_test[p,:]-des_j[q,:])) #CALCULATING THE DISTANCE
                    euc_dict[(img_list[i][0:5],img_list[j][0:5])] = distances
    return euc_dict #RETURNING ALL THE DISTANCES OF ALL THE IMAGE PAIRS

In [0]:
# CALCULATING THE IMAGE CORRESPONDENCES, ACCURACIES AND PLOTTING OF GRAPHS

# INPUTS FOR THE FUNCTION: 1. DICTIONARY CONTAINING THE EUCLIDEAN DISTANCES 2. LIST OF K's 3.LIST OF THE INPUT IMAGES
def img_corr_acc_graphs(euc_dict,k_list,img_list):
    avg_accuracies = np.zeros((len(k_list),))  # AVERAGE ACCURACIES INITIALIZATION BASED ON THE K VALUES
    img_list = np.array(img_list)              # CONVERTING THE img_list TO AN ARRAY
    for k in range(len(k_list)):
        print(k_list[k])
        accuracies = []                        #ACCURACIES OF THE INDIVIDUAL IMAGES FOR A K VALUE
        for i in range(len(img_list)):         # FOR A TEST IMAGE
            img_list_temp = np.delete(img_list,i) #REMOVE THE TEST IMAGE FROM THE img ARRAY
            #print(img_list)
            sum_dist = []                       # COLLECTING THE DISTANCES SUM OF THE COMPARING IMAGES
            for j in range(len(img_list_temp)): # A COMPARING IMAGE FROM THE REMAINING IMAGES
                if img_list[i]!=img_list_temp[j]: #JUST TO ENSURE THAT TEST AND COMPARING IMAGES ARE NOT THE SAME
                    try:
                        distances = euc_dict[(img_list[i][0:5],img_list_temp[j][0:5])] #COLLECTING THE EUCLIDEAN DISTANCES
                    except:
                        distances = euc_dict[(img_list_temp[j][0:5],img_list[i][0:5])]
                    
                    distances.sort()                   #SORTING THE DISTANCES IN ASCENDING ORDER
                    if k_list[k]=='large':             #FOR A LARGE VALUE OF K
                        distances = distances[0:]
                    else:                              #TAKING THE LEAST K DISTANCES
                        distances = distances[0:k_list[k]]
                    sum_dist.append(sum(distances))    # TAKING THE SUM OF THE EUCLIDEAN DISTANCES
            img_indices = list(np.argsort(sum_dist))   # CALCULATING THE ASCENDING ORDER INDICES
            count = 0 
            for c in range(5):                         #CALCULATING THE ACCURACY FOR THE TEST IMAGE
                if img_list[i][0:2]==img_list_temp[img_indices[c]][0:2]:
                    count = count + 1
            accuracies.append((count/5)*100)          #APPENDING THE ACCURACY FOR THE TEST IMAGE
        avg_accuracies[k] = sum(accuracies)/len(img_list) #AVG ACCURACY BASED ON THE K VALUE 
    plt.plot(k_list,avg_accuracies)                       # PLOTTING THE GRAPH
    plt.xlabel('k_values')
    plt.ylabel('accuracies')
    plt.title('average accuracies based on the k values using SIFT')
    plt.savefig('img_corr_acc_graphs.png') 
    return avg_accuracies                                 # RETURNING THE AVG ACCURACIES 
    

RUN THE CODE HERE

In [0]:
l = os.listdir('/content/') ## give the address of the data in order to get the images list
l.remove('sample_data')     ##colab had these extra files, therefore removed them 
l.remove('.config')
l.remove('.ipynb_checkpoints')
euc_dict_ = euclidean_distances(l,'/content/','SIFT')
img_corr_acc_graphs(euc_dict_,[1,5,10,20,30,40,50,60,70,80,'large'],l)

MANUAL GENERATION OF THE COMPARISION GRAPHS

FOR SIFT

In [0]:
sift_256 = [83.33333333, 83.33333333, 83.7037037 , 81.11111111, 81.11111111, #256 SIZE IMAGE ACCURACIES
       79.62962963, 78.88888889, 77.40740741, 77.03703704, 76.2962963 ,
       10.37037037]
sift_512 = [80.37037037, 82.96296296, 81.11111111, 80.        , 79.25925926, #512 SIZE IMAGE ACCURACIES
       79.25925926, 76.66666667, 75.55555556, 75.55555556, 75.18518519,
       10.37037037]
plt.plot([1,5,10,20,30,40,50,60,70,80,'large'],sift_256)
plt.plot([1,5,10,20,30,40,50,60,70,80,'large'],sift_512)
plt.xlabel('k_values')
plt.ylabel('accuracies')
plt.title('SIFT results comparision based on the size of the image')
plt.legend(['256_images','512_images'])
plt.savefig('img_comparisions.png') 

FOR ORB

In [0]:
orb_1024 = [53.33333333, 56.2962963 , 54.07407407, 54.07407407, 51.48148148, #1024 image accuracies
       48.14814815, 47.77777778, 46.66666667, 45.92592593, 44.44444444,
        4.44444444]
orb_256 = [47.03703704, 49.62962963, 48.88888889, 48.88888889, 47.40740741, #256 image accuracies
       45.55555556, 44.81481481, 44.07407407, 43.33333333, 42.59259259,
        9.62962963]
orb_512 = [55.18518519, 58.51851852, 58.51851852, 53.7037037 , 52.59259259, #512 image accuracies
       50.37037037, 48.88888889, 47.77777778, 45.92592593, 45.18518519,
        9.25925926]
k_list = [1,5,10,20,30,40,50,60,70,80,'large']
plt.plot(k_list,orb_1024,'r')
plt.plot(k_list,orb_256,'g')
plt.plot(k_list,orb_512,'b')
plt.xlabel('k_values')
plt.ylabel('accuracies')
plt.title('ORB results comparision based on the size of the image')
plt.legend(['1024_images','256_images','512_images'])
plt.savefig('img_orb_comparisions.png')
plt.show()